In [91]:
import os
import cv2 as cv
import numpy as np
from tqdm import tqdm

In [74]:
def time_to_frames(t, fps):
    return np.round(np.multiply(t, fps)).astype(int)

def frames_to_time(f, fps):
    return np.divide(f, fps)

In [95]:
def crop_video(input_video_path, output_video_path, x, y, w, h):
    # Open the input video
    cap = cv.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    original_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    out = cv.VideoWriter(output_video_path, fourcc, fps, (w, h))

    # Process each frame
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # Crop the frame
        cropped_frame = frame[y:y+h, x:x+w]

        # Write the cropped frame to the output video
        out.write(cropped_frame)

    # Release everything when job is finished
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [ ]:
input_video_path = "C0012.MP4"
output_video_path = "output_cropped.mp4"
x, y, w, h = 750, 280, 250, 150  # Specify the x, y, width, and height

crop_video(input_video_path, output_video_path, x, y, w, h)

In [94]:
def get_score_framestamps(input_video_path, output_video_path, basket_bound_x=(115, 160), basket_bound_y=(110, 115), cooldown_seconds=1):
    """
    Records a frame as a score if the ball enters the specified basket bounding box.
    The score bounding box should be aligned to the top of the basket.

    params:
        input_video_path: 
        output_video_path: 
        basket_bound_x: tuple
        basket_bound_y: tuple
        cooldown_seconds: length of time before the next detection is allowed
    """
    line_colour = (227, 73, 121)

    # Open the input video
    cap = cv.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    # removes the background using KNN algorithm, which is good if small parts of a complex background change frequently
    # https://www.reddit.com/r/opencv/comments/yrbl07/question_how_does_knn_background_subtractor_work/
    # https://en.wikipedia.org/wiki/Kernel_density_estimation 
    # tl;dr learns a background model by progressively applying each frame to the model
    fgbg = cv.createBackgroundSubtractorKNN()

    prev_seconds = -1
    framestamps = []
    # Process each frame
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # remove background
        mask = fgbg.apply(frame)
        # median blur to denoise https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
        mask = cv.medianBlur(mask, 5)

        if not np.any(mask):
            continue

        # calc ball's center of mass using pixels
        com = (np.mean(np.argwhere(mask), axis=0)).astype(int)
        cv.circle(frame, com[::-1], 3, line_colour, 2)

        cv.line(frame, (basket_bound_x[0], basket_bound_y[0]), (basket_bound_x[1], basket_bound_y[0]), line_colour, 2)
        # if com crosses the line, print timestamp
        if basket_bound_y[0] < com[0] < basket_bound_y[1] and basket_bound_x[0] < com[1] < basket_bound_x[1]:
            current_seconds = i / fps
            if not framestamps or current_seconds >= prev_seconds + cooldown_seconds:
                print(f"basket made at: {current_seconds//60}m{round(current_seconds%60)}s")
                prev_seconds = current_seconds
                framestamps.append(i)

    # Release everything when job is finished
    cap.release()
    cv.destroyAllWindows()

    print("Found ", len(framestamps), " scored baskets.")
    return framestamps

In [88]:
score_framestamps = get_score_framestamps("output_cropped.mp4", "mask_video.mp4", (115,160), (105, 115))

basket made at: 1.0m58s
basket made at: 3.0m21s
basket made at: 4.0m4s
basket made at: 4.0m52s
basket made at: 5.0m22s
basket made at: 5.0m46s
basket made at: 6.0m25s
basket made at: 6.0m38s
basket made at: 7.0m7s
basket made at: 7.0m34s
basket made at: 7.0m57s
basket made at: 8.0m26s
basket made at: 8.0m38s
basket made at: 9.0m38s
basket made at: 9.0m45s
basket made at: 9.0m59s
basket made at: 10.0m2s
basket made at: 10.0m37s
basket made at: 10.0m41s
basket made at: 10.0m46s
Found  20  scored baskets.


In [106]:
from pathlib import Path

def segment_video(input_video_path, score_framestamps, before_score_seconds=3, after_score_seconds=1, split_segments=False):
    """
    Params:
        input_video_path: path to video to segment; output highlight reel
        score_framestamps: list of frames where a basket was scored
        before_score_seconds: buffer length before the shot
        highlight_len_after_frames: buffer length after the shot
        split_segments: whether each segment should be its own video
    """
    # Open the input video
    cap = cv.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    original_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

    highlight_frames = before_score_seconds * fps
    highlight_num = 0
    out = cv.VideoWriter(f"{Path(input_video_path).stem}_highlight.mp4", fourcc, fps, (original_width, original_height))
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        if highlight_num == len(score_framestamps):
            break

        start_frame = score_framestamps[highlight_num] - highlight_frames
        if i > start_frame:
            # if reached end of highlight, move on to next highlight
            if i > start_frame + highlight_frames + after_score_seconds*fps:
                print(highlight_num)
                highlight_num += 1
                if split_segments:
                    out.release()
                    out = cv.VideoWriter(f"{Path(input_video_path).stem}_highlight_{highlight_num}.mp4", fourcc, fps, (original_width, original_height))
            else:
                out.write(frame)

    # Release everything when job is finished
    cap.release()
    out.release()

In [108]:
segment_video("C0012.MP4", score_framestamps, before_score_seconds=5, after_score_seconds=2)

 18%|█▊        | 5999/32688 [01:31<10:17, 43.24it/s]

0


 31%|███       | 10135/32688 [02:43<10:26, 36.00it/s]

1


 38%|███▊      | 12324/32688 [11:33<12:19, 27.56it/s]    

...

17


 98%|█████████▊| 32158/32688 [18:04<00:18, 28.14it/s]

18


 99%|█████████▉| 32391/32688 [18:13<00:10, 29.61it/s]

19
